데이터 분석 단순 에이전트 (Data Analysis Simple Agent)
📘 개요 (Overview)

이 튜토리얼은 자연어(Natural Language) 로 데이터셋을 해석하고 질문에 답변할 수 있는 AI 기반 데이터 분석 에이전트를 만드는 방법을 다룹니다.
이 에이전트는 언어 모델(Language Model) 과 데이터 조작 도구(Data Manipulation Tools) 를 결합하여, 사용자가 직관적으로 데이터를 탐색할 수 있도록 합니다.

💡 동기 (Motivation)

데이터 분석은 종종 전문 지식과 도구를 필요로 하기 때문에, 비전문가가 통찰을 얻기 어렵습니다.
하지만 자연어를 이해하는 AI 에이전트를 통해,

“프로그래밍이나 통계 지식 없이도 데이터를 자유롭게 탐색하고 질문할 수 있는 환경”
을 구축할 수 있습니다.

이러한 접근은 데이터 분석의 민주화(democratization) 를 가능하게 합니다 — 누구나 데이터로부터 유용한 정보를 추출할 수 있게 되는 것이죠.

🧩 핵심 구성 요소 (Key Components)
구성 요소	역할
언어 모델 (Language Model)	사용자의 자연어 질의를 이해하고 인간처럼 응답을 생성
데이터 조작 프레임워크 (Data Manipulation Framework)	데이터셋의 필터링, 집계, 계산 등 분석 작업 수행
에이전트 프레임워크 (Agent Framework)	언어 모델과 데이터 조작 도구를 연결해주는 구조
합성 데이터셋 (Synthetic Dataset)	실제 상황을 모사한 예시 데이터 (데모용)
⚙️ 방법 (Method)

합성 데이터셋 생성

자동차 판매(car sales) 데이터를 가정한 간단한 테이블을 만듭니다.
예: Brand, Year, Price, Units Sold 등의 열(column)

언어 모델과 데이터 분석 기능 결합

LLM이 “질문 → 코드 → 결과” 흐름으로 데이터를 해석하도록 설계합니다.

질의 처리 함수 구현

사용자의 자연어 질문(예: “2020년 가장 많이 팔린 브랜드는?”)을 입력으로 받아
→ 데이터를 분석하고
→ 자연어로 결과를 설명하는 함수 작성

에이전트 시연

예시 질의를 통해 모델이 데이터에 기반한 대답을 실제로 생성하는 모습을 확인합니다.

🧠 결론 (Conclusion)

이 접근 방식은 다음과 같은 이점을 제공합니다:

비전문가 접근성 향상
→ 누구나 프로그래밍 지식 없이 데이터 분석 가능

질문 처리 유연성
→ 통계, 집계, 추세, 비교 등 다양한 질문을 자연스럽게 처리

빠른 탐색
→ 즉석(ad-hoc) 데이터 탐색 및 분석 가능

🌍 의미

이 방법은 데이터 분석을 보다 쉽게 만들어,
기업과 기관이 데이터 기반 의사결정(Data-driven decision-making) 을 강화하도록 돕습니다.
즉, AI가 데이터 해석의 중간 장벽을 허물어
모든 사용자가 자연어로 데이터와 대화할 수 있는 시대를 여는 것입니다.

In [2]:
import os
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents import AgentType
from langchain_openai import ChatOpenAI
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Load environment variables
from dotenv import load_dotenv
import os

# Load environment variables and set OpenAI API key
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# Set a random seed for reproducibility
np.random.seed(42)

In [3]:
# Generate sample data
n_rows = 1000

# Generate dates
start_date = datetime(2022, 1, 1)
dates = [start_date + timedelta(days=i) for i in range(n_rows)]

# Define data categories
makes = ['Toyota', 'Honda', 'Ford', 'Chevrolet', 'Nissan', 'BMW', 'Mercedes', 'Audi', 'Hyundai', 'Kia']
models = ['Sedan', 'SUV', 'Truck', 'Hatchback', 'Coupe', 'Van']
colors = ['Red', 'Blue', 'Black', 'White', 'Silver', 'Gray', 'Green']

# Create the dataset
data = {
    'Date': dates,
    'Make': np.random.choice(makes, n_rows),
    'Model': np.random.choice(models, n_rows),
    'Color': np.random.choice(colors, n_rows),
    'Year': np.random.randint(2015, 2023, n_rows),
    'Price': np.random.uniform(20000, 80000, n_rows).round(2),
    'Mileage': np.random.uniform(0, 100000, n_rows).round(0),
    'EngineSize': np.random.choice([1.6, 2.0, 2.5, 3.0, 3.5, 4.0], n_rows),
    'FuelEfficiency': np.random.uniform(20, 40, n_rows).round(1),
    'SalesPerson': np.random.choice(['Alice', 'Bob', 'Charlie', 'David', 'Eva'], n_rows)
}

# Create DataFrame and sort by date
df = pd.DataFrame(data).sort_values('Date')

# Display sample data and statistics
print("\nFirst few rows of the generated data:")
print(df.head())

print("\nDataFrame info:")
df.info()

print("\nSummary statistics:")
print(df.describe())


First few rows of the generated data:
        Date       Make      Model  Color  Year     Price  Mileage  \
0 2022-01-01   Mercedes      Sedan  Green  2022  57952.65   5522.0   
1 2022-01-02  Chevrolet  Hatchback    Red  2021  58668.22  94238.0   
2 2022-01-03       Audi      Truck  White  2019  69187.87   7482.0   
3 2022-01-04     Nissan  Hatchback  Black  2016  40004.44  43846.0   
4 2022-01-05   Mercedes  Hatchback    Red  2016  63983.07  52988.0   

   EngineSize  FuelEfficiency SalesPerson  
0         2.0            24.7       Alice  
1         1.6            26.2         Bob  
2         2.0            28.0       David  
3         3.5            24.8       David  
4         2.5            24.1       Alice  

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            1000 non-null   datetime64[n

In [5]:
# Create the Pandas DataFrame agent
agent = create_pandas_dataframe_agent(
    ChatOpenAI(model="gpt-4o", temperature=0),
    df,
    verbose=True,
    allow_dangerous_code=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)
print("Data Analysis Agent is ready. You can now ask questions about the data.")

Data Analysis Agent is ready. You can now ask questions about the data.


In [9]:
def ask_agent(question):
    """Function to ask questions to the agent and display the response"""
    response = agent.invoke({
        "input": question,
        "agent_scratchpad": f"Human: {question}\nAI: To answer this question, I need to use Python to analyze the dataframe. I'll use the python_repl_ast tool.\n\nAction: python_repl_ast\nAction Input: ",
    })
    print(f"Question: {question}")
    print(f"Answer: {response}")
    print("---")

In [10]:
# Example questions
ask_agent("What are the column names in this dataset?")
ask_agent("How many rows are in this dataset?")
ask_agent("What is the average price of cars sold?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.columns.tolist()'}`


['Date', 'Make', 'Model', 'Color', 'Year', 'Price', 'Mileage', 'EngineSize', 'FuelEfficiency', 'SalesPerson']The column names in the dataset are:

1. Date
2. Make
3. Model
4. Color
5. Year
6. Price
7. Mileage
8. EngineSize
9. FuelEfficiency
10. SalesPerson

> Finished chain.
Question: What are the column names in this dataset?
Answer: {'input': 'What are the column names in this dataset?', 'agent_scratchpad': "Human: What are the column names in this dataset?\nAI: To answer this question, I need to use Python to analyze the dataframe. I'll use the python_repl_ast tool.\n\nAction: python_repl_ast\nAction Input: ", 'output': 'The column names in the dataset are:\n\n1. Date\n2. Make\n3. Model\n4. Color\n5. Year\n6. Price\n7. Mileage\n8. EngineSize\n9. FuelEfficiency\n10. SalesPerson'}
---


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'len(df